# Tree Alignment

As shown in [1], prior to detecting data fields we need to identify aligned regions across pages that look like potential data values. The paper uses DEPTA [2], but we are unsure if this is usable for PDF document too so let's explore it here.

## 1. Load pdf and build a tag tree out of it.

In [4]:
# Tree with defaultdict
# https://gist.github.com/hrldcpr/2012250 but implemented by
# extending a dict object:
# http://stackoverflow.com/questions/6780952/how-to-change-behavior-of-dict-for-an-instance
class Tree(dict):
    """Implementation of perl's autovivification feature."""
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

test = Tree()
test['left']['value'] = 'leftval'
test['right']['value'] = 'rightval'
test['right']['right1']['value'] = 'testval'
print(test)

{'right': {'right1': {'value': 'testval'}, 'value': 'rightval'}, 'left': {'value': 'leftval'}}


In [5]:
import os, sys, inspect
base_path = os.path.realpath(
    os.path.abspath(
        os.path.join(
            os.path.split(
                inspect.getfile(
                    inspect.currentframe()
                )
            )[0],
            '..'
        )
    )
)
sys.path.append(base_path)

class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html).decode('utf-8')

from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import PDFConverter
from pdfminer.pdfpage import PDFPage

from pdfminer.layout import LTPage, LTContainer, LTAnno
from pdfminer.layout import LTImage, LTChar
from pdfminer.layout import LTTextBox

from pdfminer.image import ImageWriter

try:
    from cStringIO import StringIO
except ImportError:
    from StringIO import StringIO
    
table = ListTable()
table.append([
    'type',
    'text',
    'x',
    'x1',
    'y',
    'y1',
    'page',
    'textbox_id',
    'textline_id'
])
tree = Tree()
types = set()
last_type = ''
class CustomPDFConverter(PDFConverter):
    def __init__(self, rsrcmgr, outfp, codec='utf-8', pageno=1, laparams=None,
                 showpageno=False, imagewriter=None):
        PDFConverter.__init__(self, rsrcmgr, outfp, codec=codec, pageno=pageno, laparams=laparams)
        self.showpageno = showpageno
        self.imagewriter = imagewriter
        self.current_total_height = 0
        return

    def write(self, message):
        self.outfp.write(message)

    def receive_layout(self, ltpage):
        """Stuff to do when layout received.

        Inherited from :class:`pdfminer.converter.PDFConverter`.

        Args:
            ltpage: :class:`pdfminer.layout.LTPage` object.
        """
        def render(item, page, page_width, current_total_height):
            """What to do when rendering certain :class:`pdfminer.layout.LT.*` objects.
            
            item.bbox has a set of (x0, y0, x1, y1)
            which we will store as part of our features.

            Important notes on features:
            - In pdf, (0,0) coordinate is positioned at bottom left, and each page has its own
              coordinates. To make it easier for learning and users to label, we combine all the
              pages into one huge, continuous plane, with position (0,0) on top left. This way
              users don't have to specify page number to create blocks, and learner can learn contents
              that span across multiple pages i.e. large paragraph of texts.

            Facts about LTTextBox:
            - It contains list of texts contained in LTTextLine... object,
              instead of single characters. Values of texts can be gathered with
              object.current_value. LTTextLine... contains LTChar.
            - LTTextLineHorizontal, LTTextLineVertical, or any child of LTTextBox does
              not have index, but we create our own index on the go (textline_id), so later
              we can trace back which LTText does an LTChar belong to.
            - LTTextBox does not seem to contain another LTTextBox, nor does it
              contain characters directly (at least in the examples so far).
            - Index can be gained from property `index` e.g. `item.index` (we will
              think about how to get this element from index later).

            Facts about LTImage:
            - object.srcsize are different from width and height calculated from bbox.
            - Belongs to no textbox.
            """
            global last_type
            if item.__class__.__name__ != last_type:
                last_type = item.__class__.__name__
                table.append([item.__class__.__name__])
            if isinstance(item, LTPage):
                page = ltpage.pageid             
            if isinstance(item, LTContainer):
                for child in item:
                    render(child, page, page_width, current_total_height)
            if isinstance(item, LTTextBox):
                for textline_id, textline in enumerate(item):
#                     for char in textline:
#                         if not isinstance(char, LTAnno):
                            element = textline
                            text = element.get_text().encode(self.codec, 'ignore')
                            tree[page][item.index][textline_id]['value'] = text
                            table.append([
                                element.__class__.__name__,
                                text,
                                element.bbox[0],
                                element.bbox[1],
                                element.bbox[2],
                                element.bbox[3],
                                page,
                                item.index,
                                textline_id
                            ])

        self.current_total_height += ltpage.height
        render(ltpage, ltpage.pageid, ltpage.width, self.current_total_height)
        return

In [6]:
outfp = sys.stdout
pagenos = set()
pdf_path = 'pdfs/test.pdf'
f = open(pdf_path, 'rb')
text = f.read()

rsrcmgr = PDFResourceManager(caching=True)
codec = 'utf-8'
# imagewriter = ImageWriter('images')

laparams = LAParams()
device = CustomPDFConverter(rsrcmgr, outfp, codec=codec, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
interpreter.debug = False

pagenos = set()
text_io = StringIO(text)

for page in PDFPage.get_pages(text_io, pagenos):
    interpreter.process_page(page)
device.close()
print("types:", types)
table

('types:', set([]))


[['type', 'text', 'x', 'x1', 'y', 'y1', 'page', 'textbox_id', 'textline_id'],
 ['LTPage'],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  '3150 Rutland Rd, Victoria, British Columbia   V8R4R8 - 348418 | Realtor.ca\n',
  53.999978399999996,
  778.9796884079999,
  425.2150016779313,
  789.8876840447999,
  1,
  0,
  0],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  'Page 1 of 3\n',
  503.639798544,
  778.9796884079999,
  557.9997768000001,
  789.8876840447999,
  1,
  1,
  0],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  '< Back\n',
  69.719972112,
  696.3885214444799,
  92.14515055792504,
  703.6965185212799,
  1,
  2,
  0],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  'http://www.realtor.ca/propertyDetails.aspx?PropertyId=15448812\n',
  53.999978399999996,
  -0.2999998800000001,
  370.95035990379677,
  10.607995756800001,
  1,
  3,
  0],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  '6/1/2015\n',
  515.279793888,
  -0.2999998800000001,
  557.9997768,
  10.607995756800001,
  1,
  4,
  0],
 ['LTLine'],
 ['LTFigure'],
 ['LTImage'],
 ['LTRect'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTFigure'],
 ['LTImage'],
 ['LTRect'],
 ['LTPage'],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  '3150 Rutland Rd, Victoria, British Columbia   V8R4R8 - 348418 | Realtor.ca\n',
  53.999978399999996,
  778.9796884079999,
  425.2150016779313,
  789.8876840447999,
  2,
  0,
  0],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  'Page 2 of 3\n',
  503.639798544,
  778.9796884079999,
  557.9997768000001,
  789.8876840447999,
  2,
  1,
  0],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  '3150 Rutland Rd, Victoria, British Columbia \xc2\xa0 V8R4R8\n',
  69.719972112,
  522.3216310712639,
  295.51692179318405,
  531.8467472612159,
  2,
  2,
  0],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  '$7,500,000\xc2\xa0\n',
  69.719972112,
  507.72699690912,
  109.38973425208879,
  514.69659412128,
  2,
  3,
  0],
 ['LTTextLineHorizontal',
  'Listing ID:\xc2\xa0348418\n',
  69.719972112,
  497.958640816464,
  122.56356835075304,
  504.579758168016,
  2,
  3,
  1],
 ['LTTextBoxHorizontal'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal'],
 ['LTChar'],
 ['LTAnno'],
 ['LTTextLineHorizontal',
  'Property Type\n',
  72.479971008,
  464.95805401670395,
  112.84874878048244,
  470.998371600576,
  2,
  4,
  0],
 ['LTTextLineHorizontal',
  'Property Type\n',
  72.239971104,
  464.95805401670395,
  112.60874887648244,
  470.998371600576,
  2,
  4,
  1],
 ['LTTextLineHorizontal',
  'Property Type\n',
  72.239971104,
  464.71805411270395,
  112.60874887648244,
  470.758371696576,
  2,
  4,
  2],
 ['LTTextLineHorizontal',
  'Property Type\n',
  72.479971008,
  464.71805411270395,
  112.84874878048244,
  470.758371696576,
  2,
  4,
  3],
 ['LTTextLineHorizontal',
  'Single Family\n',
  72.23997

In [ ]:
tree